In [31]:
import sys
import re
sys.path.append(r'C:/Users/shyam.p/Desktop/PRO/')
from  lib.initiate_ssh_connection import SSHClient
import mysql.connector
import socket
import json
import threading
import time
import requests


In [3]:
############# NEED TO FUNCTION ###################################################

# Replace these values with your actual database credentials
def sql_connect():
    host = "192.168.109.137"
    port = 3306
    user = "root"
    password = "password"
    database = "DEVICE_TRACKER"

    # Establish a connection to the database
    connection = mysql.connector.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        database=database
    )
    return connection

In [4]:
def get_dev_details():
    dev = {}
    connection = sql_connect()
    try:
    # Create a cursor to execute SQL queries
        cursor = connection.cursor()

        # Define the SQL query to retrieve IP and TAG from DEVICES table
        query = "SELECT IP, TAG FROM DEVICES"

        # Execute the query
        cursor.execute(query)

        # Fetch all the rows
        rows = cursor.fetchall()

        # Print the results
        for row in rows:
            ip, tag = row
            dev[ip] = tag

    except:
        pass
  #  cursor.close()
    connection.close()
    return dev

def get_pass_details(ip):
    connection = sql_connect()
    cursor = connection.cursor()
    query =  f"SELECT USERNAME, PASSWORD FROM DEVICES WHERE IP = %s"
    cursor.execute(query, (ip,))
    cred = cursor.fetchone()
    connection.close()
    return cred

In [5]:
def check_ping(devices):
    ping_status = dict()
    for ip, tag in devices.items():
        try:
            # Create a socket object and attempt to connect to the IP address on port 80
            sock = socket.create_connection((ip, 80), timeout=2)
            sock.close()
            ping_status[ip] = 'UP'
        except (socket.timeout, socket.error):
            ping_status[ip] = 'DOWN'
    return ping_status, [i for i in ping_status.keys() if ping_status[i]=='UP']

In [6]:
######################################## Dictonary To Json ##############################################

In [6]:
details = get_dev_details()
details

{'192.168.107.21': 'LAN-DEVICE', '192.168.107.95': 'HA-DUT-1'}

In [32]:
_,l = check_ping(details)
l

['192.168.107.21']

In [7]:
def initiate_ssh(l):
    status_check = {}
    for ip in l:
            status = {}
            username, password = get_pass_details(ip)
            obj = SSHClient(ip,username,password)
            obj.connect()
            intefaces = obj.run_execmcd("ifconfig  | awk '/^[a-zA-Z]/{print $1}'")
            intefaces = intefaces.split('\n')
            intefaces.remove('')
            intefaces.remove('lo')
            for i in intefaces:
                interface = obj.run_execmcd(f"ethtool {i} | grep 'Link detected' | awk '{{print $3}}'")
                status[i] = interface.strip()
            status_check[ip] = status
            obj.close_connection()
    return json.dumps(status_check)

In [ ]:
########################################################################### TESt #######################################################################################

In [10]:
details = get_dev_details()
details
'''Discription : This function return availabe device in the database with IP and Tag'''

{'192.168.107.21': 'LAN-DEVICE', '192.168.107.95': 'HA-DUT-1'}

In [20]:
l = check_ping(details)
l
#''' Discription : This function Returen status '''

['192.168.107.21']

In [46]:
int_status = initiate_ssh(l)
type(int_status) 

Connected to 192.168.107.21
Connection to 192.168.107.21 closed.


str

In [48]:
####################################################### Engin ###############################################
def pipe():
        sql_connect()
        try:
                data = get_dev_details()
                ping_status, up_device = check_ping(data)  #up_device is buffer
                val_json = initiate_ssh(up_device)
                ping_status = json.dumps(ping_status)
                url = "http://localhost:5000/receive_data" 
                headers = {'Content-Type': 'application/json'}
                data_to_send = {'ping_status': ping_status, 'interface_status': val_json}
                response = requests.post(url, json=data_to_send, headers=headers)
                print("Sucessfully Sended....")
                return response
        except:
                print("Failed....")
                return False
        
        
        

In [47]:
pipe()

Connected to 192.168.107.21
Connection to 192.168.107.21 closed.
{"192.168.107.21": "UP", "192.168.107.95": "DOWN"} {"192.168.107.21": {"eth0": "yes", "eth1": "yes", "eth2": "yes"}}
Sucessfully Sended....


<Response [200]>

In [20]:
ping_status, val_json = pipe()
ping_status

Connected to 192.168.107.21
Connection to 192.168.107.21 closed.


{'192.168.107.21': 'UP', '192.168.107.95': 'DOWN'}

In [65]:
stop_event = threading.Event()

def engin(interval_seconds):
    while not stop_event.is_set():
        pipe()
        time.sleep(interval_seconds)

In [66]:
def start_engin():
    interval_seconds = 10
    stop_event = threading.Event()
    engine_thread = threading.Thread(target=engin, args=(interval_seconds,))
    engine_thread.start()
    return engine_thread
    
    
def stop_engin(instance):
    stop_event.set()   
    instance.join()

In [67]:
ins = start_engin()

Connected to 192.168.107.21
Connection to 192.168.107.21 closed.
{"192.168.107.21": "UP", "192.168.107.95": "DOWN"} {"192.168.107.21": {"eth0": "yes", "eth1": "yes", "eth2": "yes"}}
Sucessfully Sended....
Connected to 192.168.107.21
Connection to 192.168.107.21 closed.
{"192.168.107.21": "UP", "192.168.107.95": "DOWN"} {"192.168.107.21": {"eth0": "yes", "eth1": "yes", "eth2": "yes"}}
Sucessfully Sended....
Connected to 192.168.107.21
Connection to 192.168.107.21 closed.
{"192.168.107.21": "UP", "192.168.107.95": "DOWN"} {"192.168.107.21": {"eth0": "yes", "eth1": "yes", "eth2": "yes"}}
Sucessfully Sended....
Connected to 192.168.107.21
Connection to 192.168.107.21 closed.
{"192.168.107.21": "UP", "192.168.107.95": "DOWN"} {"192.168.107.21": {"eth0": "yes", "eth1": "yes", "eth2": "yes"}}
Sucessfully Sended....
Connected to 192.168.107.21
Connection to 192.168.107.21 closed.
{"192.168.107.21": "UP", "192.168.107.95": "DOWN"} {"192.168.107.21": {"eth0": "yes", "eth1": "yes", "eth2": "yes"}

In [68]:
stop_engin(ins)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\shyam.p\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
